# Copyright

<PRE>
Jelen iPython notebook a Budapesti Műszaki és Gazdaságtudományi Egyetemen tartott 
"Deep Learning a gyakorlatban Python és LUA alapon" tantárgy segédanyagaként készült a   
https://github.com/bradleypallen/keras-movielens-cf oldalon található forrás alapján.

A tantárgy honlapja: http://smartlab.tmit.bme.hu/oktatas-deep-learning 
Deep Learning kutatás: http://smartlab.tmit.bme.hu/deep-learning

A forráskódot MIT licensz védi.
</PRE>

In [1]:
import pandas as pd
import numpy as np
from CFModel import CFModel # letölthető innen: https://github.com/bradleypallen/keras-movielens-cf/blob/master/CFModel.py

Using TensorFlow backend.


## Konstansok (előre elkészített adatállományok és a beágyazás dimenziószáma), valamint egy előtanított modell


In [2]:
RATINGS_CSV_FILE = 'ml1m_ratings.csv'
USERS_CSV_FILE = 'ml1m_users.csv'
MOVIES_CSV_FILE = 'ml1m_movies.csv'
MODEL_WEIGHTS_FILE = 'ml1m_weights.h5'
EMBEDDING_DIM = 120

## MovieLens 1M betöltése

In [3]:
ratings = pd.read_csv(RATINGS_CSV_FILE, sep='\t', encoding='latin-1', usecols=['userid', 'movieid', 'rating'])
max_userid = ratings['userid'].drop_duplicates().max()
max_movieid = ratings['movieid'].drop_duplicates().max()
print (len(ratings), 'értékelés betöltve.')

1000209 értékelés betöltve.


In [4]:
users = pd.read_csv(USERS_CSV_FILE, sep='\t', encoding='latin-1', usecols=['userid', 'gender', 'zipcode', 'age_desc', 'occ_desc'])
print (len(users), 'felhasználó a', max_userid, 'felhasználó közül betöltve.')

6040 felhasználó a 6040 felhasználó közül betöltve.


In [9]:
movies = pd.read_csv(MOVIES_CSV_FILE, sep='\t', encoding='latin-1', usecols=['movieid', 'title', 'genre'])
print (len(movies), 'film a', max_movieid, 'film közül betöltve.')

3883 film a 3952 film közül betöltve.


## Filmajánlás egy véletlenszerűen választott felhasználónak
A felhasználói értékelést jelezzük előre a filmekre.

In [5]:
# keras.layers.embeddings.Embedding(input_dim, output_dim, init='uniform', input_length=None, W_regularizer=None, activity_regularizer=None, W_constraint=None, mask_zero=False, weights=None, dropout=0.0)
# 2 ágon torténik közös embedding: userekre és itemekre, K_FACTORS dimenzióra
trained_model = CFModel(max_userid, max_movieid, EMBEDDING_DIM)

/home/deep/Letöltések/vitmav45-master/2016/11/CFModel.py:25: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  self.add(Merge([P, Q], mode='dot', dot_axes=1)) # skaláris szorzat, maga az értékelés


In [6]:
# 2 súlymátrixra van szükségünk, egy user->embeddingre és egy item->embeddingre
trained_model.load_weights(MODEL_WEIGHTS_FILE)

In [7]:
# Egy véletlen felhasználó és adatai
test_user=np.random.randint(0,max_userid)
users[users['userid'] == test_user]

,userid,gender,zipcode,age_desc,occ_desc
3432,3433,M,55113,18-24,programmer


In [10]:
# Egy véletlen film és adatai
test_movie=np.random.randint(0,max_movieid)
movies[movies['movieid'] == test_movie]

,movieid,title,genre
2394,2463,Ruthless People (1986),Comedy


In [11]:
# Ez a metódus model.predict-et futtat, indexelést átképezzük (adatbázisban 1-től indul)
def predict_rating(userid, movieid):
    return trained_model.rate(userid - 1, movieid - 1)

In [13]:
# Betöltjük a célfelhazsnálónktól származó felhasználói értékeléseket
user_ratings = ratings[ratings['userid'] == test_user][['userid', 'movieid', 'rating']]
# Hozzáadunk egy ajánló oszlopot és minden filmre generáljuk, hogy mennyire passzol a felhasználónk ízléséhez
user_ratings['prediction'] = user_ratings.apply(lambda x: predict_rating(test_user, x['movieid']), axis=1)
# Kiiratáshoz a predikciós és filmes táblákat egyesítjük a filmazonosító (ez közös) mentén
# Első 10-et írjuk ki
user_ratings.sort_values(by='rating', 
                         ascending=False).merge(movies, 
                                                on='movieid', 
                                                how='inner', 
                                                suffixes=['_u', '_m']).head(12) #tail(12) az utolsó 12-t írná ki

,userid,movieid,rating,prediction,title,genre
78,3433,2162,1,1.414796,"NeverEnding Story II: The Next Chapter, The (1...",Adventure|Children's|Fantasy
79,3433,1981,1,1.064095,Friday the 13th Part VIII: Jason Takes Manhatt...,Horror
80,3433,1556,1,1.139310,Speed 2: Cruise Control (1997),Action|Romance|Thriller
81,3433,1562,1,1.225156,Batman & Robin (1997),Action|Adventure|Crime
82,3433,1974,1,2.016993,Friday the 13th (1980),Horror
83,3433,1590,1,1.346338,Event Horizon (1997),Action|Mystery|Sci-Fi|Thriller
84,3433,1739,1,1.034750,3 Ninjas: High Noon On Mega Mountain (1998),Action|Children's
85,3433,2706,1,3.144991,American Pie (1999),Comedy
86,3433,153,1,1.943306,Batman Forever (1995),Action|Adventure|Comedy|Crime
87,3433,2381,1,0.937374,Police Academy 4: Citizens on Patrol (1987),Comedy


In [15]:
# Szűrés azokra a filmekre, amelyeket a felhasználónk még nem látott
recommendations = ratings[ratings['movieid'].isin(user_ratings['movieid']) == False][['movieid']].drop_duplicates()
# Ajánlás a még nem látott filmekre
recommendations['prediction'] = recommendations.apply(lambda x: predict_rating(test_user, x['movieid']), axis=1)
# Kiírjuk az első 10-et
recommendations.sort_values(by='prediction',
                          ascending=False).merge(movies,
                                                 on='movieid',
                                                 how='inner',
                                                 suffixes=['_u', '_m']).head(10)

,movieid,prediction,title,genre
0,1234,4.818746,"Sting, The (1973)",Comedy|Crime
1,1036,4.602067,Die Hard (1988),Action|Thriller
2,858,4.563595,"Godfather, The (1972)",Action|Crime|Drama
3,457,4.563026,"Fugitive, The (1993)",Action|Thriller
4,1262,4.532127,"Great Escape, The (1963)",Adventure|War
5,3421,4.517402,Animal House (1978),Comedy
6,2762,4.504848,"Sixth Sense, The (1999)",Thriller
7,1250,4.493625,"Bridge on the River Kwai, The (1957)",Drama|War
8,1221,4.462417,"Godfather: Part II, The (1974)",Action|Crime|Drama
9,1954,4.438286,Rocky (1976),Action|Drama


In [16]:
# Illetve amelyeket jó szívvel nem ajánlanánk...
recommendations.sort_values(by='prediction',
                          ascending=False).merge(movies,
                                                 on='movieid',
                                                 how='inner',
                                                 suffixes=['_u', '_m']).tail(10)

,movieid,prediction,title,genre
3608,3212,0.007456,Born to Win (1971),Drama
3609,3291,0.003394,Trois (2000),Thriller
3610,286,-0.006884,Nemesis 2: Nebula (1995),Action|Sci-Fi|Thriller
3611,843,-0.014439,Lotto Land (1995),Drama
3612,139,-0.015735,Target (1995),Action|Drama
3613,2619,-0.016119,Mascara (1999),Drama
3614,774,-0.026956,Wend Kuuni (God's Gift) (1982),Drama
3615,402,-0.029621,Open Season (1996),Comedy
3616,865,-0.038349,Small Faces (1995),Drama
3617,1630,-0.049691,"Lay of the Land, The (1997)",Comedy|Drama
